In [90]:
pip install faiss-cpu langchain langchain_community

You should consider upgrading via the 'c:\Users\KIIT\AppData\Local\Programs\Python\Python38\python.exe -m pip install --upgrade pip' command.


In [91]:
import pandas as pd
from langchain.docstore.document import Document
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

In [92]:
df=pd.read_csv("C:/Users/KIIT/OneDrive/Desktop/FINANCE_AI/Finance_prediction/Loan_default.csv")

In [93]:
df.head()

,LoanID,Age,Income,LoanAmount,CreditScore,MonthsEmployed,NumCreditLines,InterestRate,LoanTerm,DTIRatio,Education,EmploymentType,MaritalStatus,HasMortgage,HasDependents,LoanPurpose,HasCoSigner,Default
0,I38PQUQS96,56,85994,50587,520,80,4,15.23,36,0.44,Bachelor's,Full-time,Divorced,Yes,Yes,Other,Yes,0
1,HPSK72WA7R,69,50432,124440,458,15,1,4.81,60,0.68,Master's,Full-time,Married,No,No,Other,Yes,0
2,C1OZ6DPJ8Y,46,84208,129188,451,26,3,21.17,24,0.31,Master's,Unemployed,Divorced,Yes,Yes,Auto,No,1
3,V2KKSFM3UN,32,31713,44799,743,0,3,7.07,24,0.23,High School,Full-time,Married,No,No,Business,No,0
4,EY08JDHTZP,60,20437,9139,633,8,4,6.51,48,0.73,Bachelor's,Unemployed,Divorced,No,Yes,Auto,No,0


In [94]:
df = df.drop("LoanID", axis=1)
df.head()

,Age,Income,LoanAmount,CreditScore,MonthsEmployed,NumCreditLines,InterestRate,LoanTerm,DTIRatio,Education,EmploymentType,MaritalStatus,HasMortgage,HasDependents,LoanPurpose,HasCoSigner,Default
0,56,85994,50587,520,80,4,15.23,36,0.44,Bachelor's,Full-time,Divorced,Yes,Yes,Other,Yes,0
1,69,50432,124440,458,15,1,4.81,60,0.68,Master's,Full-time,Married,No,No,Other,Yes,0
2,46,84208,129188,451,26,3,21.17,24,0.31,Master's,Unemployed,Divorced,Yes,Yes,Auto,No,1
3,32,31713,44799,743,0,3,7.07,24,0.23,High School,Full-time,Married,No,No,Business,No,0
4,60,20437,9139,633,8,4,6.51,48,0.73,Bachelor's,Unemployed,Divorced,No,Yes,Auto,No,0


In [95]:
# Check for missing values and handle them
print(df.isnull().sum())

Age               0
Income            0
LoanAmount        0
CreditScore       0
MonthsEmployed    0
NumCreditLines    0
InterestRate      0
LoanTerm          0
DTIRatio          0
Education         0
EmploymentType    0
MaritalStatus     0
HasMortgage       0
HasDependents     0
LoanPurpose       0
HasCoSigner       0
Default           0
dtype: int64


In [96]:
# Impute LoanPurpose with the mode
df['LoanPurpose'].fillna(df['LoanPurpose'].mode()[0], inplace=True)

# Impute HasCoSigner with the mode
df['HasCoSigner'].fillna(df['HasCoSigner'].mode()[0], inplace=True)


# Handle missing values in the target variable 'Default'
# Option 1: Drop rows with missing target values (if very few)
df.dropna(subset=['Default'], inplace=True)

# Verify that there are no more missing values
print(df.isnull().sum())

Age               0
Income            0
LoanAmount        0
CreditScore       0
MonthsEmployed    0
NumCreditLines    0
InterestRate      0
LoanTerm          0
DTIRatio          0
Education         0
EmploymentType    0
MaritalStatus     0
HasMortgage       0
HasDependents     0
LoanPurpose       0
HasCoSigner       0
Default           0
dtype: int64


In [97]:
df = df.iloc[:1500,:]
df.shape

(1500, 17)

In [98]:
df.shape

(1500, 17)

In [99]:
documents = []
for i, row in df.iterrows():
    document = f"id:{i} | Customer_Behavior: {row.to_dict()} | Default_Status: {row['Default']}"
    documents.append(Document(page_content=document))

from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Split documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = text_splitter.split_documents(documents)

# Generate embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Build FAISS index
faiss_index = FAISS.from_documents(docs, embeddings)
faiss_index.save_local('docs/faiss_rag/')  # This will create index.faiss & index.pkl


print("FAISS vector store created and saved successfully.")


FAISS vector store created and saved successfully.


In [79]:
# Define the function to create a prompt
def create_prompt(data_point):
    prompt = f"""
    Predict the default probability for the following loan applicant:

    - Age: {data_point['Age']}
    - Income: {data_point['Income']}
    - Loan Amount: {data_point['LoanAmount']}
    - Credit Score: {data_point['CreditScore']}
    - Months Employed: {data_point['MonthsEmployed']}
    - Number of Credit Lines: {data_point['NumCreditLines']}
    - Interest Rate: {data_point['InterestRate']}
    - Loan Term: {data_point['LoanTerm']}
    - Debt-to-Income Ratio: {data_point['DTIRatio']}
    - Education: {data_point['Education']}
    - Employment Type: {data_point['EmploymentType']}
    - Marital Status: {data_point['MaritalStatus']}
    - Has Mortgage: {data_point['HasMortgage']}
    - Has Dependents: {data_point['HasDependents']}
    - Loan Purpose: {data_point['LoanPurpose']}
    - Has Co-Signer: {data_point['HasCoSigner']}
    """
    return prompt


In [19]:
# Create a list to store prompts and responses
data_list = []

In [20]:
# Generate prompts and responses
for index, row in df.iterrows():
    prompt = create_prompt(row)
    response = f"Default Probability: {row['Default']}"
    data_list.append((prompt, response))


In [ ]:
data_list[0]

("\n    Predict the default probability for the following loan applicant:\n\n    - Age: 56\n    - Income: 85994\n    - Loan Amount: 50587\n    - Credit Score: 520\n    - Months Employed: 80\n    - Number of Credit Lines: 4\n    - Interest Rate: 15.23\n    - Loan Term: 36\n    - Debt-to-Income Ratio: 0.44\n    - Education: Bachelor's\n    - Employment Type: Full-time\n    - Marital Status: Divorced\n    - Has Mortgage: Yes\n    - Has Dependents: Yes\n    - Loan Purpose: Other\n    - Has Co-Signer: Yes\n    ",
 'Default Probability: 0')

In [ ]:
from langchain.docstore.document import Document

documents = []

# Iterate over rows using .rows() method
for i, row_tuple in df.iterrows():
    document = f"id:{i}\Customer_Behavior: {row_tuple[1]}\Default_Status: {row_tuple[0]}"
    documents.append(Document(page_content=document))

In [ ]:
!pip install langchain_community

In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings
hg_embeddings = HuggingFaceEmbeddings()

C:\Users\KIIT\AppData\Local\Temp\ipykernel_32132\999870367.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  hg_embeddings = HuggingFaceEmbeddings()
C:\Users\KIIT\AppData\Local\Temp\ipykernel_32132\999870367.py:2: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  hg_embeddings = HuggingFaceEmbeddings()
c:\Users\KIIT\AppData\Local\Programs\Python\Python38\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplic

In [ ]:
!pip install chromadb

  Using cached chromadb-0.6.3-py3-none-any.whl.metadata (6.8 kB)
  Using cached build-1.2.2.post1-py3-none-any.whl.metadata (6.5 kB)
  Using cached chroma_hnswlib-0.7.6.tar.gz (32 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached posthog-3.11.0-py2.py3-none-any.whl.metadata (2.9 kB)
  Using cached onnxruntime-1.20.1-cp312-cp312-win_amd64.whl.metadata (4.7 kB)
  Using cached opentelemetry_exporter_otlp_proto_grpc-1.29.0-py3-none-any.whl.metadata (2.2 kB)
  Using cached opentelemetry_instrumentation_fastapi-0.50b0-py3-none-any.whl.metadata (2.1 kB)
  Using cached PyPika-0.48.9-py2.py3-none-any.whl
  Using cached overrides-7.7.0-py3-none-any.whl.metadata (5.8 kB)
  Using cached importlib_resourc

  error: subprocess-exited-with-error
  
  × Building wheel for chroma-hnswlib (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [5 lines of output]
      running bdist_wheel
      running build
      running build_ext
      building 'hnswlib' extension
      error: Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for chroma-hnswlib
ERROR: Failed to build installable wheels for some pyproject.toml based projects (chroma-hnswlib)


In [ ]:
!pip install langchain-groq

In [27]:
from langchain.vectorstores import Chroma

persist_directory = 'docs/chroma_rag/'

langchain_chroma = Chroma.from_documents(
    documents=documents,
    collection_name="default_prediction",
    embedding=hg_embeddings,
    persist_directory=persist_directory
)

ValueError: Batch size 1500 exceeds maximum batch size 166

In [51]:
from google.colab import userdata
# LLM with function call
from langchain_groq import ChatGroq
import os
groq_api_key=userdata.get('groq_api_key')
os.environ["GROQ_API_KEY"]=groq_api_key

In [52]:
llm=ChatGroq(groq_api_key=groq_api_key,model_name='deepseek-r1-distill-llama-70b')
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7dbf5a5f5ad0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7dbf5a5f1490>, model_name='deepseek-r1-distill-llama-70b', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [54]:
from IPython.display import display, Markdown
from langchain.schema import HumanMessage

def colorize_text(text):
    for word, color in zip(["Reasoning", "Question", "Answer", "Total time"], ["blue", "red", "green", "magenta"]):
        text = text.replace(f"{word}:", f"\n\n**{word}:**")
    return text


question = "Please explain Default in Credit Risk?"
# Wrap the question in a HumanMessage object and put it in a list
response = llm([HumanMessage(content=question)])

full_response =  f"Question: {question}\nAnswer: {response.content}" # Access the content of the AIMessage
display(Markdown(colorize_text(full_response)))



**Question:** Please explain Default in Credit Risk?


**Answer:** <think>
Okay, so I need to understand what "Default" means in the context of credit risk. I remember hearing the term before, but I'm not entirely sure about the specifics. Let me try to break it down.

First, I think default has something to do with when someone or a company can't pay back a loan. That makes sense because credit risk is about the chance that someone won't pay back what they owe. So, if a borrower defaults, they fail to meet their payment obligations. But I'm not sure about the exact definition or the different types of defaults.

I remember reading somewhere about sovereign default, which I think is when a country can't pay its debts. That must be a big deal because countries borrowing money is a significant event. Then there's corporate default, which is when companies can't pay back their bonds or loans. I guess that's why companies sometimes go bankrupt.

I also recall something about strategic default, which I think is when individuals, like homeowners, decide not to pay their mortgages even though they can afford to. That seems a bit different because it's more of a choice rather than an inability to pay.

There are different types of defaults, like technical default, where maybe the borrower misses a payment but can still pay later. Or maybe it's when they violate some terms of the loan agreement without necessarily missing a payment. I'm not too clear on that.

I'm also trying to remember how default rates work. I think it's the percentage of borrowers who default within a certain period. So, if a lender has a 2% default rate, that means 2 out of every 100 borrowers didn't pay back their loans. That must be important for lenders to assess risk and set interest rates.

Consequences of default seem severe. For the borrower, it probably hurts their credit score and makes it harder to get loans in the future. For lenders, it means they lose money, which could affect their business. Investors who buy bonds or other debt instruments also face losses if there's a default.

I'm a bit fuzzy on the difference between default and other terms like delinquency. I think delinquency is when a payment is late, but not necessarily a full default. So, delinquency might be a precursor to default if the borrower doesn't catch up on payments.

I'm also trying to piece together how credit risk is managed. I know that lenders use credit scores and other assessments to determine the likelihood of default. They might also require collateral or charge higher interest rates to borrowers who are more likely to default.

Wait, so if a borrower defaults, what happens next? I think there are processes like foreclosure for mortgages or repossession for loans like cars. For businesses, it might lead to bankruptcy, where assets are sold off to pay creditors.

I'm also wondering about the impact on the economy. If many defaults happen at once, like during a financial crisis, it could cause widespread problems for banks and other financial institutions, leading to a credit crunch. That sounds like what happened in 2008 with the housing market.

I should also consider how investors handle default risk. They might diversify their portfolios to minimize the impact of any single default. Credit rating agencies play a role here too by assigning ratings that reflect the likelihood of default, helping investors make informed decisions.

But I'm still a bit confused about the exact mechanisms. For example, how is a default formally declared? Is it as soon as a payment is missed, or is there a grace period? I think it varies depending on the loan terms. Some agreements might allow a few late payments before considering it a default.

Also, what about the difference between hard default and soft default? I think a hard default is when the borrower can't pay, while a soft default might be due to other reasons, like strategic default. Or maybe it's when the borrower is in a difficult financial situation but hasn't yet missed a payment.

I'm trying to remember if there are legal implications. Defaulting on a loan is a breach of contract, so lenders can take legal action to recover their money. This might involve suing the borrower or seizing assets. For businesses, it can lead to legal proceedings like Chapter 11 bankruptcy in the US, which is about restructuring debt rather than liquidating assets.

I'm also thinking about how default affects the lender's balance sheet. If a loan defaults, the lender has to write it off as a loss, which can affect their capital and ability to lend in the future. That's why lenders have to maintain reserves to cover potential defaults.

Another aspect is credit risk models. I know that models like Moody's or Standard & Poor's use various factors to predict the probability of default. These models help in pricing loans and bonds accurately, reflecting the risk involved.

I'm trying to recall if there are different stages before a default occurs. Maybe first, the borrower misses a payment, then goes into delinquency, and if they can't resolve it, it leads to default. Each stage might have different implications and procedures.

I also wonder about the role of collateral in reducing default risk. If a loan is secured by collateral, the lender can seize it if the borrower defaults, minimizing losses. That's why mortgage loans are considered less risky than unsecured loans like credit cards.

But even with collateral, the process can be lengthy and costly. Foreclosing on a house, for example, takes time and legal fees, which are additional costs for the lender beyond just the loan amount.

I'm also thinking about how interest rates are set. Lenders charge higher interest to borrowers with a higher probability of default to compensate for the increased risk. This is why credit card rates are so high compared to mortgages, as they're unsecured and have higher default rates.

In summary, default in credit risk is when a borrower fails to meet their payment obligations. It can happen at various levels—sovereign, corporate, or individual. There are different types like strategic default and technical default. The consequences are severe for both borrowers and lenders, affecting credit scores, financial stability, and economic conditions. Managing credit risk involves assessing the likelihood of default through credit scores, collateral, and setting appropriate interest rates. Legal and financial mechanisms are in place to handle defaults, but they can be complex and costly.
</think>

Default in credit risk refers to the failure of a borrower to meet their payment obligations, which can occur at various levels, including sovereign, corporate, or individual. Here's a structured overview:

### Types of Default:
1. **Sovereign Default**: Occurs when a country fails to meet its debt obligations.
2. **Corporate Default**: Happens when a company defaults on bonds or loans, potentially leading to bankruptcy.
3. **Strategic Default**: When an individual chooses not to pay despite having the means, often seen in mortgages.
4. **Technical Default**: Involves violating loan terms without missing payments, such as covenant breaches.

### Mechanisms and Implications:
- **Default Rate**: The percentage of borrowers defaulting within a specific period, crucial for lenders to assess risk.
- **Consequences**:
  - **For Borrowers**: Damaged credit scores, reduced access to future credit.
  - **For Lenders**: Financial loss, potentially affecting their lending capacity.
  - **For Investors**: Losses on investments like bonds.

### Processes and Management:
- **Delinquency vs. Default**: Delinquency is late payment, while default is failure to pay after a grace period.
- **Legal and Recovery Processes**: Includes foreclosure, repossession, or legal action to recover losses.
- **Credit Risk Management**: Lenders use credit scores, collateral, and interest rates to mitigate risk.

### Economic Impact:
- **Systemic Risk**: Widespread defaults can trigger financial crises, affecting entire economies.
- **Investor Strategies**: Diversification and reliance on credit ratings to manage risk exposure.

### Financial and Legal Aspects:
- **Collateral**: Reduces risk but involves costs like foreclosure processes.
- **Interest Rates**: Higher rates for riskier borrowers to compensate for potential defaults.

In essence, default is a critical aspect of credit risk management, influencing financial stability and economic health through various mechanisms and consequences.

In [55]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_community.llms import HuggingFaceHub
from IPython.display import display, Markdown
import os
import warnings
warnings.filterwarnings('ignore')


# Define the prompt template
template = """
You are an Credit Risk Expert in Financial Text Data, Analyse them and Predict is the Given Customer is going to Default or not?. If you don't know the answer, just say "Sorry, I Don't Know."
Question: {question}
Context: {context}
Answer:
"""
PROMPT = PromptTemplate(input_variables=["context", "query"], template=template)

# Ensure llm and langchain_chroma are properly initialized
retriever = langchain_chroma.as_retriever(search_kwargs={"k": 1})

qa_chain = RetrievalQA.from_chain_type(
    llm, retriever=retriever, chain_type_kwargs={"prompt": PROMPT}
)

# Define your question
# question = "The company reported inflated revenues by including sales that never occurred."
question = "\n    Predict the default probability for the following loan applicant:\n\n    - Age: 56\n    - Income: 85994\n    - Loan Amount: 50587\n    - Credit Score: 520\n    - Months Employed: 80\n    - Number of Credit Lines: 4\n    - Interest Rate: 15.23\n    - Loan Term: 36\n    - Debt-to-Income Ratio: 0.44\n    - Education: Bachelor's\n    - Employment Type: Full-time\n    - Marital Status: Divorced\n    - Has Mortgage: Yes\n    - Has Dependents: Yes\n    - Loan Purpose: Other\n    - Has Co-Signer: Yes\n    "
# question = "Revenue was recognized prematurely before the actual sales occurred."
# question = "The balance sheet provides a true and fair view of the company’s financial position."

# Run the QA chain
try:
    result = qa_chain({"query": question})
    display(result)
except RuntimeError as e:
    print(f"RuntimeError encountered: {e}")

{'query': "\n    Predict the default probability for the following loan applicant:\n\n    - Age: 56\n    - Income: 85994\n    - Loan Amount: 50587\n    - Credit Score: 520\n    - Months Employed: 80\n    - Number of Credit Lines: 4\n    - Interest Rate: 15.23\n    - Loan Term: 36\n    - Debt-to-Income Ratio: 0.44\n    - Education: Bachelor's\n    - Employment Type: Full-time\n    - Marital Status: Divorced\n    - Has Mortgage: Yes\n    - Has Dependents: Yes\n    - Loan Purpose: Other\n    - Has Co-Signer: Yes\n    ",
 'result': "<think>\nOkay, so I need to predict whether this loan applicant is going to default based on the given information. Let's break down each piece of data and see how it contributes to the overall risk.\n\nFirst, the age is 56. I'm not sure how much age impacts default risk, but older individuals might have more financial stability, but they could also be closer to retirement, which might affect their ability to repay if something goes wrong.\n\nThe income is $85,